In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df_fake = pd.read_csv('Fake.csv')
df_fake.head(3)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"


In [3]:
df_true = pd.read_csv('True.csv')
df_true.head(3)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"


In [4]:
df_fake['label'] = 0
df_true['label'] = 1

In [5]:
df = pd.concat([df_fake, df_true], ignore_index=True)

In [6]:
df.head(3)

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0


In [7]:
df.drop(['date', 'subject'], axis= 1, inplace=True, errors='ignore')

In [8]:
df = df.sample(frac=1).reset_index(drop=True)

In [9]:
df.head()

,title,text,label
0,U.S. Treasury's Mnuchin to bring up defense on...,WASHINGTON (Reuters) - U.S. Treasury Secretary...,1
1,HYSTERICAL! HILLARY’S SPEECH HIJACKED By Conse...,Hillary Clinton tried to say everyone supporti...,0
2,Thai junta says to allow parties to get ready ...,BANGKOK (Reuters) - Thai Prime Minister Prayut...,1
3,U.S.-led anti-Islamic State coalition says Ira...,BAGHDAD (Reuters) - The U.S.-led anti-Islamic ...,1
4,"Under pressure from Trump, Price resigns as he...",WASHINGTON (Reuters) - U.S. Health and Human S...,1


In [10]:
df['label'].value_counts()

label
0    23481
1    21417
Name: count, dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   44898 non-null  object
 1   text    44898 non-null  object
 2   label   44898 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [12]:
df.isnull().sum()

title    0
text     0
label    0
dtype: int64

In [13]:
df.sample(5)

,title,text,label
24426,Insight: Emails show how Republicans lobbied t...,(This version of the November 3 story officia...,1
12055,White House: no sign of collusion in campaign ...,WASHINGTON (Reuters) - The indictments of two ...,1
3488,"DAY AFTER DALLAS COPS’ MEMORIAL, Obama Invites...","When America elected a Community Agitator, the...",0
25232,Factbox: Iraq's Kurds to vote in historic refe...,"ERBIL/LONDON, Iraq (Reuters) - About five mill...",1
3415,ELEMENTARY SCHOOL PLANS ‘BLACKS ONLY’ FIELD TR...,Because there wouldn t be any outrage over a ...,0


In [14]:
df['content'] = df['title'] + " " + df['text']
df.drop(['title', 'text'], axis = 1, inplace= True)

In [15]:
df.head(3)

,label,content
0,1,U.S. Treasury's Mnuchin to bring up defense on...
1,0,HYSTERICAL! HILLARY’S SPEECH HIJACKED By Conse...
2,1,Thai junta says to allow parties to get ready ...


In [16]:
# clean Text
import re
def clean_text(text):
    text = text.lower()                                # Lowercase
    text = re.sub(r'\[.*?\]', '', text)                # Remove text in brackets
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>+', '', text)                 # Remove HTML tags
    text = re.sub(r'[^a-z\s]', '', text)               # Remove punctuation/numbers
    text = re.sub(r'\s+', ' ', text).strip()           # Remove extra whitespace
    return text

In [17]:
df['content'] = df['content'].apply(clean_text)

In [18]:
df.head()

,label,content
0,1,us treasurys mnuchin to bring up defense on ot...
1,0,hysterical hillarys speech hijacked by conserv...
2,1,thai junta says to allow parties to get ready ...
3,1,usled antiislamic state coalition says iraqiku...
4,1,under pressure from trump price resigns as hea...


In [19]:
df.isnull().sum()

label      0
content    0
dtype: int64

In [20]:
df['label'].value_counts()

label
0    23481
1    21417
Name: count, dtype: int64

## TF-IDF Vectorization

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
x = tfidf_vectorizer.fit_transform(df['content'])

In [26]:
y = df['label']

In [27]:
x.shape

(44898, 211040)

## Train Test Split

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [31]:
x_train.shape, y_train.shape

((35918, 211040), (35918,))

## Build and Train Model

In [37]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [39]:
mnb = MultinomialNB()
mnb.fit(x_train, y_train) 

MultinomialNB()

In [40]:
mnb.score(x_train, y_train)*100, mnb.score(x_test, y_test)*100

(95.18625758672532, 94.7327394209354)

In [41]:
y_prd_mnb = mnb.predict(x_test)

In [42]:
accuracy_score(y_test, y_prd_mnb)

0.9473273942093541

In [43]:
classification_report(y_test, y_prd_mnb)

'              precision    recall  f1-score   support\n\n           0       0.95      0.95      0.95      4696\n           1       0.94      0.95      0.94      4284\n\n    accuracy                           0.95      8980\n   macro avg       0.95      0.95      0.95      8980\nweighted avg       0.95      0.95      0.95      8980\n'

In [45]:
confusion_matrix(y_test, y_prd_mnb)

array([[4449,  247],
       [ 226, 4058]])

## PassiveAggressiveClassifier

In [46]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [47]:
pac = PassiveAggressiveClassifier(max_iter=1000)
pac.fit(x_train, y_train)

PassiveAggressiveClassifier()

In [48]:
pac.score(x_train, y_train)*100, pac.score(x_test, y_test)*100

(100.0, 99.543429844098)

In [49]:
confusion_matrix(y_test, pac.predict(x_test))

array([[4676,   20],
       [  21, 4263]])

## Summary Table

In [51]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [52]:
# naive bayes
nb_accuracy= accuracy_score(y_test, mnb.predict(x_test))
nb_precision= precision_score(y_test, mnb.predict(x_test)) 
nb_recall= recall_score(y_test, mnb.predict(x_test)) 
nb_f1=f1_score(y_test, mnb.predict(x_test))

In [53]:
# Passive Aggregation
pa_accuracy = accuracy_score(y_test, pac.predict(x_test))
pa_precision = precision_score(y_test, pac.predict(x_test)) 
pa_recall = recall_score(y_test, pac.predict(x_test)) 
pa_f1 = f1_score(y_test, pac.predict(x_test))

In [55]:
comparision_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'], 
    'Naive Bayes': [nb_accuracy, nb_precision, nb_recall, nb_f1],
    'Passive Aggregation': [pa_accuracy, pa_precision, pa_recall, pa_f1]
})

In [56]:
comparision_df

,Metric,Naive Bayes,Passive Aggregation
0,Accuracy,0.947327,0.995434
1,Precision,0.942625,0.995330
2,Recall,0.947246,0.995098
3,F1 Score,0.944930,0.995214


## Save The Model

In [57]:
import joblib

In [58]:
joblib.dump(pac, 'fake_news_detector_model.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

## Load The Model

In [60]:
model = joblib.load('fake_news_detector_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

In [61]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [69]:
def predict_news(news_list):
    results = []
    for news in news_list:
        cleaned = clean_text(news)
        vectorized = vectorizer.transform([cleaned])
        prediction = model.predict(vectorized)[0]
        label = "📰 Real News ✅" if prediction == 1 else "🚨 Fake News ❌"
        results.append((news, label))
    return results

In [82]:
sample_news = [
    'Scientists discovered dragons living under the Himalayans.',
    "The stock market saw a 10% rise due to improved economic indicators.",
    "Breaking: Stock market surges as inflation slows down.",
    "NASA confirms that aliens landed in Kathmandu last night.",
    "A secret lab found a cure for death, officials confirm."
]

In [83]:
predicted = predict_news(sample_news)

for i, (text, result) in enumerate(predicted, 1):
    print(f"{i}. {result}\n   -> {text}\n")

1. 🚨 Fake News ❌
   -> Scientists discovered dragons living under the Himalayans.

2. 📰 Real News ✅
   -> The stock market saw a 10% rise due to improved economic indicators.

3. 🚨 Fake News ❌
   -> Breaking: Stock market surges as inflation slows down.

4. 🚨 Fake News ❌
   -> NASA confirms that aliens landed in Kathmandu last night.

5. 🚨 Fake News ❌
   -> A secret lab found a cure for death, officials confirm.

